In [1]:
from bdsg.bdsg import HashGraph
from bdsg.bdsg import SnarlDistanceIndex
from bdsg.bdsg import PackedGraph

In [2]:
graph = HashGraph()
seq = ["CGA", "TTGG", "CCGT", "C", "GT", "GATAA", "CGG", "ACA", "GCCG", "ATATAAC"]
#seq = ["AAA", "BBB", "CCC", "DDD"]
n = []
for s in seq:
    n.append(graph.create_handle(s))

In [3]:
graph.create_edge(n[0],n[1])
graph.create_edge(n[1],n[2])
graph.create_edge(n[2],n[3])
graph.create_edge(n[2],n[4])
graph.create_edge(n[3],n[5])
graph.create_edge(n[4],n[5])
graph.create_edge(n[5],n[8])
graph.create_edge(n[5],n[6])
graph.create_edge(n[6],n[7])
graph.create_edge(n[6],n[8])
graph.create_edge(n[7],n[9])
graph.create_edge(n[8],n[9])
graph.create_edge(n[8],n[5])


# graph.create_edge(n[0],n[1])
# graph.create_edge(n[0],n[2])
# graph.create_edge(n[0],n[3])

In [4]:
neighbors = []
def gather_neighbors(neighbor_handle):
    neighbors.append(neighbor_handle)
    return True  # Continue traversing edges

# Step 5: Follow the outgoing (right side) edges
graph.follow_edges(n[5], go_left=False, iteratee=gather_neighbors)


for nb in neighbors:
    print(f'{graph.get_sequence(nb)}')
    

GCCG
CGG


In [5]:
def next_node_list(handle):
    lis = []
    graph.follow_edges(handle, False, lambda y: lis.append(y) or True)
    return lis

print(f'n0: {graph.get_sequence(n[0])}')
next_node = next_node_list(n[0])[0]
print(f'n1: {graph.get_sequence(next_node)}')
next_node = next_node_list(next_node)[0]
print(f'n2: {graph.get_sequence(next_node)}')
next_nodes = next_node_list(n[2])
for el in next_nodes:
    print(f'{graph.get_sequence(el)}')

n0: CGA
n1: TTGG
n2: CCGT
C
GT


In [6]:
path = graph.create_path_handle("path_test")
graph.append_step(path, n[0])
graph.append_step(path, n[1])
graph.append_step(path, n[2])
graph.append_step(path, n[4])
graph.append_step(path, n[5])
graph.append_step(path, n[6])
graph.append_step(path, n[7])
graph.append_step(path, n[9])

In [7]:
step = graph.path_begin(path)

while (graph.has_next_step(step)):
    current_node_handle = graph.get_handle_of_step(step)
    print(graph.get_sequence(current_node_handle))
    step = graph.get_next_step(step)
    
current_node_handle = graph.get_handle_of_step(step)

print(graph.get_sequence(current_node_handle))

CGA
TTGG
CCGT
GT
GATAA
CGG
ACA
ATATAAC


In [8]:
with open("./test.gfa","w") as f:
    print(f'S\t1\tAAAA', file=f)
    print(f'S\t2\tBB', file=f)
    print(f'S\t3\tC', file=f)
    print(f'S\t4\tDDDDD', file=f)
    print(f'L\t1\t+\t2\t+\t0M', file=f)
    print(f'L\t1\t+\t3\t+\t0M', file=f)
    print(f'L\t2\t+\t4\t+\t0M', file=f)
    print(f'L\t3\t+\t4\t+\t0M', file=f)
    print(f'W\tP1\t1\tchr1\t0\t10\t>1>2>4', file=f)
    print(f'W\tP2\t2\tchr1\t0\t9\t>1>3>4', file=f)
    

In [9]:
s_idx = SnarlDistanceIndex()
s_idx.deserialize('./index.dist')

RuntimeError: Could not load from file ./index.dist: No such file or directory

In [ ]:
root_handle = s_idx.get_root()

In [ ]:
def snarl_iteratee(snarl_handle):
    print("Snarl:", s_idx.net_handle_as_string(snarl_handle))
    return True

def chain_iteratee(chain_handle):
    print("Chain:", s_idx.net_handle_as_string(chain_handle))
    return True

def node_iteratee(node_handle):
    print("Node:", s_idx.net_handle_as_string(node_handle))
    return True


In [ ]:
s_idx.traverse_decomposition(snarl_iteratee, chain_iteratee, node_iteratee)

In [ ]:
gr = PackedGraph()
gr.deserialize('test.vg')
node_id = 2
handle = gr.get_handle(node_id, is_reverse=False)

In [ ]:
n_handle = s_idx.get_net(handle, gr)

In [ ]:
s_idx.is_node(n_handle)

In [ ]:
parent_handle = s_idx.get_parent(n_handle)
p2_handle = s_idx.get_parent(parent_handle)
if s_idx.is_chain(parent_handle):
    print(f"Node {node_id} is part of the chain represented by handle: {parent_handle}")
elif s_idx.is_snarl(parent_handle):
    print(f"Node {node_id} is inside the snarl represented by handle: {parent_handle}")
else:
    print(f"Error: Could not find parent chain or snarl for node {node_id}")


In [ ]:
s_idx.is_node(p2_handle)

In [ ]:
s_idx.is_snarl(p2_handle)

In [ ]:
# Step 4: Traverse the chain
def visit_chain_child(child_net_handle):
    if s_idx.is_node(child_net_handle):
        print(f"Child is a node: {s_idx.node_id(child_net_handle)}")
    elif s_idx.is_snarl(child_net_handle):
        print(f"Child is a snarl: {s_idx.net_handle_as_string(child_net_handle)}")
    return True  # Continue traversing

# Traverse the chain and visit every child in it
s_idx.for_each_child(s_idx.get_parent(p2_handle), visit_chain_child)

In [ ]:
node_id = 1
handle = gr.get_handle(node_id, is_reverse=False)

In [ ]:
steps_on_node = []

gr.for_each_step_on_handle(handle, lambda y: steps_on_node.append(y) or True)

In [ ]:
len(steps_on_node)

In [ ]:
path_handles = []
for step in steps_on_node:
    path_handle = gr.get_path_handle_of_step(step)
    path_handles.append(path_handle)

In [ ]:
path_names = []
for path_handle in path_handles:
    path_name = gr.get_path_name(path_handle)
    path_names.append(path_name)

In [ ]:
path_names

In [ ]:
s_idx.is_chain(parent_handle)

In [ ]:
s_idx.chain_minimum_length(parent_handle)

In [ ]:
s_idx.minimum_length(p2_handle)